<h2>Purely classical

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path

%config InlineBackend.figure_format = 'retina'
sns.set_theme(style="whitegrid", context="talk")

# Load data (works whether run from repo root or test/)
data_path = Path('../data/input_data.csv')

df = pd.read_csv(data_path)
scenario_cols = [c for c in df.columns if c.startswith('scenario_')]

# Scenario aggregates
df['mean_wind'] = df[scenario_cols].mean(axis=1)
df['p10_wind'] = df[scenario_cols].quantile(0.10, axis=1)
df['p90_wind'] = df[scenario_cols].quantile(0.90, axis=1)

df.head()

,hour,scenario_1,scenario_2,scenario_3,scenario_4,scenario_5,scenario_6,scenario_7,scenario_8,scenario_9,scenario_10,scenario_11,scenario_12,scenario_13,price,mean_wind,p10_wind,p90_wind
0,1,9.7879,11.5034,11.9791,14.1908,12.8853,15.5571,6.6945,2.5541,1.2603,1.0340,1.1279,1.1258,1.1785,88.96,6.990669,1.12622,13.92970
1,2,9.8956,11.5903,12.0019,14.2250,12.7664,15.4913,7.0044,2.7490,1.2407,1.2346,1.2413,1.3511,1.1126,83.82,7.069554,1.23582,13.93328
2,3,9.9362,11.6674,12.0415,14.2240,12.6339,15.3941,7.2759,2.9433,1.3617,1.5136,1.4085,1.6377,1.0718,83.00,7.162277,1.37106,13.90598
3,4,9.9593,11.6214,12.0927,14.1792,12.4775,15.2231,7.5040,3.1699,1.5604,1.8890,1.6405,1.9333,1.0590,82.56,7.254562,1.57642,13.83886
4,5,9.9773,11.4704,12.2061,14.1143,12.3181,14.9783,7.7236,3.4366,1.7483,2.2889,1.8440,2.1095,1.0803,82.82,7.330438,1.76744,13.75506


In [ ]:
# hours = df['hour'].astype(int).tolist()
# prices = df['price'].astype(float).tolist()

# prices
hours = df['hour'].astype(int).tolist()
prices = dict(zip(df['hour'], df['price']))

deterministic_demand = 
net_demand = 

{(1, 1): np.float64(9.7879),
 (1, 2): np.float64(11.5034),
 (1, 3): np.float64(11.9791),
 (1, 4): np.float64(14.1908),
 (1, 5): np.float64(12.8853),
 (1, 6): np.float64(15.5571),
 (1, 7): np.float64(6.6945),
 (1, 8): np.float64(2.5541),
 (1, 9): np.float64(1.2603),
 (1, 10): np.float64(1.034),
 (1, 11): np.float64(1.1279),
 (1, 12): np.float64(1.1258),
 (1, 13): np.float64(1.1785),
 (2, 1): np.float64(9.8956),
 (2, 2): np.float64(11.5903),
 (2, 3): np.float64(12.0019),
 (2, 4): np.float64(14.225),
 (2, 5): np.float64(12.7664),
 (2, 6): np.float64(15.4913),
 (2, 7): np.float64(7.0044),
 (2, 8): np.float64(2.749),
 (2, 9): np.float64(1.2407),
 (2, 10): np.float64(1.2346),
 (2, 11): np.float64(1.2413),
 (2, 12): np.float64(1.3511),
 (2, 13): np.float64(1.1126),
 (3, 1): np.float64(9.9362),
 (3, 2): np.float64(11.6674),
 (3, 3): np.float64(12.0415),
 (3, 4): np.float64(14.224),
 (3, 5): np.float64(12.6339),
 (3, 6): np.float64(15.3941),
 (3, 7): np.float64(7.2759),
 (3, 8): np.float64(2.94

In [ ]:
import pulp

hours = df['hour'].astype(int).tolist()
prices = dict(zip(df['hour'], df['price']))

# Battery parameters (from the statement)
power_charge_max = 5.0   # MW
power_discharge_max = 4.0  # MW
energy_cap = 16.0        # MWh
eta_c = 0.80             # charging efficiency
eta_d = 1.0              # discharging efficiency
max_cycles = 2           # full cycles/day cap

# Constant term: expected wind revenue across scenarios (equiprobable)
expected_wind_revenue = (df['mean_wind'] * df['price']).sum()

# Model
model = pulp.LpProblem('BatteryArbitrage', pulp.LpMaximize)

# Variables to optimize
charge = pulp.LpVariable.dicts('charge', hours, lowBound=0)
discharge = pulp.LpVariable.dicts('discharge', hours, lowBound=0)
soc = pulp.LpVariable.dicts('soc', [0] + hours, lowBound=0, upBound=energy_cap)
is_charge = pulp.LpVariable.dicts('is_charge', hours, lowBound=0, upBound=1, cat='Binary')

# Constraints 

# Initial state
model += soc[0] == 0

for t in hours:
    model += soc[t] == soc[t-1] + eta_c * charge[t] - eta_d * discharge[t]
    model += charge[t] <= power_charge_max * is_charge[t]
    model += discharge[t] <= power_discharge_max * (1 - is_charge[t])

# End-of-day condition
model += soc[hours[-1]] == 0

# Cycle-budget constraint (limits total discharged energy)
model += pulp.lpSum(discharge[t] for t in hours) <= max_cycles * energy_cap

# Objective: battery arbitrage + constant wind revenue
battery_revenue = pulp.lpSum([prices[t] * (discharge[t] - charge[t]) for t in hours])
model += battery_revenue + expected_wind_revenue

model.solve(pulp.PULP_CBC_CMD(msg=0))

status = pulp.LpStatus[model.status]
battery_profit = pulp.value(battery_revenue)
total_revenue = pulp.value(model.objective)

print('Solver status:', status)
print(f"Expected wind revenue: ?{expected_wind_revenue:,.0f}")
print(f"Battery profit:        ?{battery_profit:,.0f}")
print(f"Total portfolio:       ?{total_revenue:,.0f}")


Solver status: Optimal
Expected wind revenue: ?17,645
Battery profit:        ?2,035
Total portfolio:       ?19,680
